In [12]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import csv
import math
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import pytorch_lightning as pl
import glob

## First, we slice the train images into 31 x 31 pixels with the ground truth in the middle

In [13]:
def ndigit(n, x):
    x = str(x)
    while(len(x) < n):
        x = "0" + x
    return x

In [78]:
def load_data(res, files = 20):
    j = 0
    path = ["02", "train"]
    res = int((res-1)/2)
    
    for p in path:
        for f in range(files):
            image = np.load(f"images_{p}/images/image_{ndigit(3, f)}.npy")
            mask = np.load(f"masks_{p}/masks/mask_{ndigit(3, f)}.npy")
            image = np.reshape(image, (1024,1024,10))
            mask = np.reshape(mask, (1024,1024,1))

            # Add padding to every image (and mask) edge in case there are ground truths which are too close to an edge
            padded_image = np.pad(image, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')
            padded_mask = np.pad(mask, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')

            # Extract ground truths
            ground_truths_pos = np.array(np.where(padded_mask != 0)).T
            
            # Slice and save patches around each ground truth
            for i in ground_truths_pos: 
                patch = (padded_image[i[0]-res : i[0]+res+1, i[1]-res : i[1]+res+1, :], padded_mask[i[0], i[1], 0])
                np.save(f"patches/train/patch_{p}_{ndigit(3, f)}_{ndigit(5, j)}.npy", np.array(patch, dtype="object"))                                 
                j += 1

In [79]:
# Check number of ground truths
# pos = 0
# for i in range(20):
#     mask = np.load(f"masks_02/masks/mask_{ndigit(3, i)}.npy")
#     ground_truths_pos = np.array(np.where(mask != 0)).T
#     pos = pos + len(ground_truths_pos)
# print(pos)

In [80]:
res = 15
load_data(res)

## Then, we load the data and have a look

In [81]:
batch_size = 128

In [82]:
# Load patches
directory = 'patches/train'
file_paths = glob.glob(directory + '/*.npy')
trainset0 = [np.load(file_path, allow_pickle=True) for file_path in file_paths]
trainset = []
for pic in trainset0:
    trainset.append(pic)
len(trainset)

38863

In [97]:
trainset[0]

array([array([[[1968, 2409, 2409, ..., 2564, 2564, 3006],
               [3006, 3047, 3047, ..., 2631, 2631, 2165],
               [2165, 2307, 2307, ..., 2619, 2619, 2545],
               ...,
               [2543, 2499, 2499, ..., 2129, 2129, 2124],
               [2124, 2121, 2121, ..., 2239, 2239, 2223],
               [2223, 2232, 2232, ..., 2008, 2008, 2101]],

              [[2305, 2245, 2245, ..., 2123, 2123, 2160],
               [2160, 2176, 2176, ..., 2997, 2997, 2350],
               [2350, 2226, 2226, ..., 1983, 1983, 2088],
               ...,
               [2057, 2103, 2103, ..., 2527, 2527, 2737],
               [2737, 2847, 2847, ..., 2684, 2684, 2288],
               [2288, 2296, 2296, ..., 2282, 2282, 2629]],

              [[2355, 2573, 2573, ..., 2291, 2291, 2307],
               [2307, 2341, 2341, ..., 3254, 3254, 2674],
               [2674, 2364, 2364, ..., 2120, 2120, 1991],
               ...,
               [2243, 2289, 2289, ..., 2372, 2372, 2389],
        

In [84]:
def enrich_channels(trainset, veggie, moisture):
    #trainset[pic_no][0][h][w][channel] -> pixel value
    print(f"Shape vorher: Liste mit ({res},{res},10) Bildern")
    counter = 0
    trainset = trainset.copy()  # Make a copy of the trainset

    if veggie:
        pic_no = 0
        for pic in trainset:
            counter += 1
            pixel_values = pic[0]
            channel8 = pixel_values[:, :, 7]
            channel4 = pixel_values[:, :, 3]
            channels = pic[0].shape[2]
            width = pic[0].shape[0]
            height = pic[0].shape[1]

            vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(channel8, channel4))
            trainset_transformed = np.concatenate((trainset[pic_no][0], vegetation_array[:, :, np.newaxis]), axis=2)
            trainset[pic_no] = (trainset_transformed, trainset[pic_no][1])
            pic_no += 1

        print("Added Vegetation (B8-B4)/(B8+B4)")

    if moisture:
        pic_no = 0
        for pic in trainset:
            pixel_values = pic[0]
            channel8a = pixel_values[:, :, 7]
            channel11 = pixel_values[:, :, 8]
            channels = pic[0].shape[2]
            width = pic[0].shape[0]
            height = pic[0].shape[1]

            moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(channel8a, channel11))
            trainset_transformed = np.concatenate((trainset[pic_no][0], moisture_array[:, :, np.newaxis]), axis=2)
            trainset[pic_no] = (trainset_transformed, trainset[pic_no][1])
            pic_no += 1

        print("Added Moisture (B8A-B11)/(B8A+B11)")
        
    print("shape nachher", trainset[0][0].shape)  # Print the shape of the first item

    return trainset

In [85]:
trainset_enriched = enrich_channels(trainset, True, True)
len(trainset_enriched)
#trainset[pic_no][0][h][w][channel] -> pixel value
#trainset[pic_no][1] -> Ground truth 

Shape vorher: Liste mit (15,15,10) Bildern


C:\Users\damian.garrell\AppData\Local\Temp\ipykernel_8284\950853731.py:18: RuntimeWarning: invalid value encountered in divide
  vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(channel8, channel4))


Added Vegetation (B8-B4)/(B8+B4)


C:\Users\damian.garrell\AppData\Local\Temp\ipykernel_8284\950853731.py:35: RuntimeWarning: invalid value encountered in divide
  moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(channel8a, channel11))


Added Moisture (B8A-B11)/(B8A+B11)
shape nachher (15, 15, 12)


38863

In [87]:
X,y = trainset_enriched[0]
X.shape

(15, 15, 12)

In [88]:
class CustomDataset(Dataset):
    def __init__(self, trainset, transform):
        self.trainset = trainset
        self.transform = transform

    def __len__(self):
        return len(self.trainset)

    def __getitem__(self, index):
        data, target = self.trainset[index]

        if self.transform:
            data = self.transform(data)

        return data, target

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.ConvertImageDtype(torch.float),
     transforms.Lambda(lambda x : x / 3000),
     transforms.Lambda(lambda x : torch.where(x > 1, 1, x)), # clip images between 0 and 1
     transforms.Normalize(mean=(0.5,)*12,
                          std=(0.5,)*12)
     ])

In [89]:
# Create the custom dataset
trainset_transformed = CustomDataset(trainset_enriched, transform=transform)

len(trainset_transformed)

38863

In [98]:
trainset_transformed[0]

(tensor([[[ 0.3120,  1.0000,  0.4433,  ...,  0.6953,  0.4160,  0.4820],
          [ 0.5367,  0.4400,  0.5667,  ...,  0.3713,  0.8247,  0.5253],
          [ 0.5700,  0.5380,  0.7827,  ...,  0.4953,  0.5927,  0.7140],
          ...,
          [-0.2993,  0.1533,  0.2647,  ...,  0.8307,  0.3927,  0.7853],
          [-0.1027, -0.4427,  0.4147,  ...,  0.8400,  0.4167,  0.7027],
          [ 0.0267,  0.3267,  0.4553,  ...,  0.5680,  0.8320,  0.8347]],
 
         [[ 0.6060,  1.0000,  0.5380,  ...,  0.6660,  0.4140,  0.4880],
          [ 0.4967,  0.4507,  0.4840,  ...,  0.4020,  0.8980,  0.5307],
          [ 0.7153,  0.5607,  0.5760,  ...,  0.5260,  0.6073,  0.7180],
          ...,
          [-0.0973, -0.0553,  0.3647,  ...,  0.8127,  0.4787,  1.0000],
          [-0.0293, -0.5313,  0.5627,  ...,  0.8633,  0.4113,  0.6807],
          [-0.0720,  0.4507,  0.4160,  ...,  0.6033,  0.8420,  0.8073]],
 
         [[ 0.6060,  1.0000,  0.5380,  ...,  0.6660,  0.4140,  0.4880],
          [ 0.4967,  0.4507,

In [90]:
# Calculate the sizes of the training set and validation set
train_size = int(0.8 * len(trainset_transformed))
val_size = len(trainset_transformed) - train_size

# Split trainset into trainset and valset
trainset_load, valset_load = random_split(trainset_transformed, [train_size, val_size])
print(len(trainset_load), len(valset_load))

# Create data loaders for the training set and validation set
trainloader = DataLoader(trainset_load, batch_size=batch_size, shuffle=True, num_workers=0)
validloader = DataLoader(valset_load, batch_size=batch_size, shuffle=True, num_workers=0)


## Next, we define the model and train it

In [94]:
class MyCNNModel(pl.LightningModule): # New! def init(self, layers, lr=0.01, classes=None): super().init() # <- Very important! self.lr = lr self.classes = classes ## Build model self.layers = nn.Sequential(layers) # Create a sequential model

    def __init__(self, *layers, classes=None):
        super().__init__()

        self.lr = 0.01  # Assign the learning rate here
        self.classes = classes

        self.layers = nn.Sequential(*layers)  # Create a sequential model
        
    def forward(self, X):
        return self.layers(X)

    def predict(self, X):
        with torch.no_grad():
            y_hat = self(X).argmax(1)
        if self.classes is not None:
            y_hat = [self.classes[i] for i in y_hat]
        return y_hat

    def training_step(self, batch, batch_idx, log_prefix='train'): # New !
        X, y = batch
        y_hat = self(X)
        loss_fn = nn.MSELoss()
        loss = loss_fn(y_hat, y)
        self.log(f"{log_prefix}_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx): # New!
        with torch.no_grad():
            return self.training_step(batch, batch_idx, log_prefix='valid')

    def configure_optimizers(self):
        # Adam with Weight Decay (Most commonly used)
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

        # Simplest scheduler is ReduceLROnPlateau. This scheduler reduces the learning rate by 0.1
        # if the val_loss has not decreased within the last 10 epochs.
        scheduler = {
            # REQUIRED: The scheduler instance
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True),
            # The unit of the scheduler's step size, could also be 'step'.
            # 'epoch' updates the scheduler on epoch end whereas 'step'
            # updates it after a optimizer update.
            "interval": "epoch",
            # How many epochs/steps should pass between calls to
            # `scheduler.step()`. 1 corresponds to updating the learning
            # rate after every epoch/step.
            "frequency": 1,
            # Metric to to monitor for schedulers like `ReduceLROnPlateau`
            "monitor": "val_loss",
            # If set to `True`, will enforce that the value specified 'monitor'
            # is available when the scheduler is updated, thus stopping
            # training if not found. If set to `False`, it will only produce a warning
            "strict": True,
            # If using the `LearningRateMonitor` callback to monitor the
            # learning rate progress, this keyword can be used to specify
            # a custom logged name
            "name": None,
        }
        return {"optimizer": optimizer, 'lr-scheduler': scheduler}

## Implement model

In [95]:
# Implements entry to SepConv2d, see Lang et al. (2019), p. 6
class MyEntryLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.out_channels = out_channels

        self.proj_out = nn.Conv2d(in_channels, out_channels[len(out_channels)-1], (1,1))

        self.entry_blocks = nn.ModuleList()
        for i in range(len(out_channels)):
            self.entry_blocks.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels[i], (1, 1)),
                nn.BatchNorm2d(out_channels[i]),
                nn.ReLU()
            ))
            in_channels = out_channels[i]  # Update in_channels for next iteration

    def forward(self, x):
        x_entry = x
        for i in range(len(self.out_channels)):
            x_entry = self.entry_blocks[i](x_entry)
        x = self.proj_out(x)
        return (x + x_entry)

In [96]:
# Implements SepConv2D
class MySepConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel, **kwargs):
        super().__init__()
        if in_channels == out_channels:
            self.proj_out = nn.Identity()
        else:
            self.proj_out = nn.Conv2d(in_channels, out_channels, (1,1), **kwargs)

        self.sep_conv_block = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, kernel, groups=in_channels, **kwargs), # depthwise SepConv
            nn.Conv2d(in_channels, out_channels, (1,1), **kwargs), # pointwise SepConv
            nn.BatchNorm2d(out_channels)
        )
    
    def forward(self, x):
        x_sep_conv = self.sep_conv_block(x)
        x_sep_conv_2 = self.sep_conv_block(x_sep_conv) # performs second SepConv, see Lang et al. (2019), p. 6
        x = self.proj_out(x)
        return (x + x_sep_conv_2) # adds original input and sep_conv_2 output

In [99]:
tree_model = MyCNNModel(
    MyEntryLayer(12, [128, 256, 512]), # increase number of channels to 512
    MySepConvLayer(512, 512, (3,3), padding='same'),
    MySepConvLayer(512, 512, (3,3), padding='same'),
    MySepConvLayer(512, 512, (3,3), padding='same'),
    nn.AdaptiveMaxPool2d(1),
    nn.Flatten(1),
    nn.Linear(512, 1)
)

In [100]:
# New, we need a trainer class
from pytorch_lightning.callbacks import RichProgressBar, RichModelSummary
trainer1 = pl.Trainer(devices=1, accelerator="cpu", precision='64', max_epochs=1,
                      callbacks=[RichProgressBar(refresh_rate=50),
                                 RichModelSummary(3),
                                ])

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\damian.garrell\AppData\Local\anaconda3\envs\venv_dl\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [101]:
trainer1.fit(tree_model, trainloader, validloader)

c:\Users\damian.garrell\AppData\Local\anaconda3\envs\venv_dl\lib\site-packages\pytorch_lightning\core\optimizer.py:357: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'lr-scheduler'}
  rank_zero_warn(


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                    ┃ Type              ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ layers                  │ Sequential        │  981 K │
│ 1  │ layers.0                │ MyEntryLayer      │  174 K │
│ 2  │ layers.0.proj_out       │ Conv2d            │  6.7 K │
│ 3  │ layers.0.entry_blocks   │ ModuleList        │  168 K │
│ 4  │ layers.1                │ MySepConvLayer    │  268 K │
│ 5  │ layers.1.proj_out       │ Identity          │      0 │
│ 6  │ layers.1.sep_conv_block │ Sequential        │  268 K │
│ 7  │ layers.2                │ MySepConvLayer    │  268 K │
│ 8  │ layers.2.proj_out       │ Identity          │      0 │
│ 9  │ layers.2.sep_conv_block │ Sequential        │  268 K │
│ 10 │ layers.3                │ MySepConvLayer    │  268 K │
│ 11 │ layers.3.proj_out       │ Identity          │      0 │
│ 12 │ layers.3.sep_conv_block │ Sequential        │  268 K │
│ 13 │ layers.4                │ AdaptiveMaxPool2d │      0 │
│ 14 │ layers.5                │ Flatten           │      0 │
│ 15 │ layers.6                │ Linear            │    513 │
└────┴─────────────────────────┴───────────────────┴────────┘

Trainable params: 981 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 981 K                                                                                                
Total estimated model params size (MB): 3

Output()

c:\Users\damian.garrell\AppData\Local\anaconda3\envs\venv_dl\lib\site-packages\pytorch_lightning\trainer\connectors
\data_connector.py:480: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly 
recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(

c:\Users\damian.garrell\AppData\Local\anaconda3\envs\venv_dl\lib\site-packages\pytorch_lightning\trainer\connectors
\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may 
be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on
this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

c:\Users\damian.garrell\AppData\Local\anaconda3\envs\venv_dl\lib\site-packages\torch\nn\modules\loss.py:536: 
UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). 
This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

c:\Users\damian.garrell\AppData\Local\anaconda3\envs\venv_dl\lib\site-packages\pytorch_lightning\trainer\connectors
\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may
be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on
this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

In [35]:
tree_model.eval()
tree_model = tree_model.float()
batch = next(iter(trainloader))
inputs = batch[0]
inputs = inputs.float()

print(inputs.shape)


with torch.no_grad():
    predictions = tree_model(inputs)


print("Predictions:", predictions)


torch.Size([128, 12, 15, 15])
forward: torch.Size([128, 12, 15, 15])
Predictions: tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
   